<a href="https://colab.research.google.com/github/Shubham-Kanse/Misogyny-Detection-with-API-Based-Data-Augmentation/blob/main/Misogyny_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentence-transformers
!pip install scikit-learn
!pip install emoji

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/drive/MyDrive/Mysogyny_Detection/data/final_labels.csv'
df = pd.read_csv(file_path)

# --- Column setup ---
TEXT_COLUMN = 'body'
RAW_LABEL_COLUMN = 'level_1'
LABEL_COLUMN = 'misogynistic_binary_label'  # 0 for non-misogynistic, 1 for misogynistic

# --- Step 1: Clean missing values ---
print(f"Missing in '{TEXT_COLUMN}': {df[TEXT_COLUMN].isnull().sum()}")
print(f"Missing in '{RAW_LABEL_COLUMN}': {df[RAW_LABEL_COLUMN].isnull().sum()}")

df[TEXT_COLUMN] = df[TEXT_COLUMN].replace(r'^\s*$', np.nan, regex=True)
df.dropna(subset=[TEXT_COLUMN, RAW_LABEL_COLUMN], inplace=True)

# --- Step 2: Check unique label values ---
print("\nUnique values in 'level_1':")
print(df[RAW_LABEL_COLUMN].unique())

# --- Step 3: Create binary labels ---
# Based on actual values in your dataset
df[LABEL_COLUMN] = df[RAW_LABEL_COLUMN].apply(
    lambda x: 0 if str(x).strip().lower() == 'nonmisogynistic' else 1
)

# --- Step 4: Confirm label creation ---
print("\nBinary Label distribution:")
print(df[LABEL_COLUMN].value_counts())

# --- Step 5: Ensure label column is int type ---
df[LABEL_COLUMN] = df[LABEL_COLUMN].astype(int)

# --- Step 6: Create data splits ---
df_train = df[df['split'] == 'train']
df_val   = df[df['split'] == 'val']
df_test  = df[df['split'] == 'test']

# --- Step 7: Show test set distribution ---
print("\nTest Set Label Distribution:")
print(df_test[LABEL_COLUMN].value_counts())

# --- Optional: Quick check ---
print(f"\nTotal samples: {len(df)}")
print(f"Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

print("\nSample text example from test set:")
print(df_test[[TEXT_COLUMN, RAW_LABEL_COLUMN, LABEL_COLUMN]].head())


Missing in 'body': 12
Missing in 'level_1': 0

Unique values in 'level_1':
['Nonmisogynistic' 'Misogynistic']

Binary Label distribution:
misogynistic_binary_label
0    5856
1     699
Name: count, dtype: int64

Test Set Label Distribution:
misogynistic_binary_label
0    1172
1     129
Name: count, dtype: int64

Total samples: 6555
Train: 5254, Val: 0, Test: 1301

Sample text example from test set:
                                                 body          level_1  \
2   Honestly my favorite thing about this is that ...  Nonmisogynistic   
3                Source? Doesnt sound right to me idk  Nonmisogynistic   
9                      Isn't this the plot of Cocoon?  Nonmisogynistic   
15  Professionals say, that dehydration is caused ...  Nonmisogynistic   
17     *I can't believe it's not* virgina spread open  Nonmisogynistic   

    misogynistic_binary_label  
2                           0  
3                           0  
9                           0  
15                        

In [ ]:
import emoji
import re

def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text) # Ensure text is string

    # Emoji decoding
    text = emoji.demojize(text, delimiters=(" ", " ")) # "😃" -> " smiling_face_with_big_eyes "

    # Lowercasing
    text = text.lower()

    # Simple cleaning: remove URLs, mentions (if any), special characters (optional, BERT can handle some)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@\w+', '', text) # Remove mentions
    text = re.sub(r'#', '', text) # Remove hashtag symbol but keep the text
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation (optional, consider if it helps your specific dataset)
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace

    return text

# Apply preprocessing to the text column
df['processed_text'] = df[TEXT_COLUMN].apply(preprocess_text)

print("\nSample of processed text:")
print(df['processed_text'].head())


Sample of processed text:
0    do you have the skin of a 80 year old grandma ...
1    this is taking a grain of truth and extrapolat...
2    honestly my favorite thing about this is that ...
3                  source doesnt sound right to me idk
4    damn i saw a movie in which the old woman bath...
Name: processed_text, dtype: object


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
# We'll fit it on the non-misogynistic and misogynistic content separately if we want to find class-specific keywords,
# or on all content if we want general keywords for each document.
# For augmenting each document, we'll fit on the current document or a small batch.
# Here, let's extract keywords for each document individually.
# This is a simplification; the paper might imply a more global TF-IDF for identifying

def extract_keywords_for_doc(doc_text, top_n=5):
    if not doc_text.strip(): # Handle empty strings
        return []
    try:
        tfidf_vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_vectorizer.fit([doc_text]) # Fit on the single document
        feature_names = tfidf_vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_vectorizer.transform([doc_text]).toarray().flatten()
        sorted_indices = tfidf_scores.argsort()[::-1]
        keywords = [feature_names[i] for i in sorted_indices[:top_n] if tfidf_scores[i] > 0]
        return keywords
    except ValueError: # Happens if doc_text is empty after stopword removal
        return []


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load the Sentence-Transformer model
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

def contextual_expansion(text, keywords, model, top_k_similar=1):
    """
    Expands text by finding terms semantically similar to its keywords.
    This is a simplified interpretation of the paper's "contextual expansion".
    The paper mentions "retrieve semantically related words and phrases" [cite: 134]
    and "identifies related misogynistic concepts"[cite: 136].

    A more direct approach for augmentation might be paraphrasing or synonym replacement.
    Here, we find similar words to the keywords and append them.
    This is a challenging step to replicate exactly without more details on how
    "related concepts" were integrated.

    Let's try a simple approach: for each keyword, find a similar term from a predefined
    candidate list or by perturbing the keyword slightly and finding similar phrases.
    Given the paper talks about "discrimination against women" as a related concept[cite: 136],
    it suggests a knowledge base or a way to generate candidates.

    Alternative simple augmentation: For each keyword, we could try to find a highly similar
    word from a general vocabulary, or just use the keywords themselves to reinforce the text.
    The paper's description is: "The model then identifies related misogynistic concepts...
    based on the semantic proximity of their embeddings"[cite: 136].

    Let's try to augment by adding keywords themselves, as a very basic first step,
    and then consider how to get "related concepts". A full replication of finding
    "sexism", "discrimination against women" from "misogyny" [cite: 135, 136] via embeddings alone
    without a target corpus of concepts is non-trivial.

    A practical approach for augmenting text for BERT:
    If we have keywords, we can append them. For more sophisticated augmentation,
    techniques like back-translation or using a thesaurus with embeddings are common.
    The paper's phrasing "feed the model with feature patterns" [cite: 131] might mean
    the keywords themselves are the "focused set of terms" for the sentence transformer
    to create an embedding of the *context* of those keywords, and then similar *sentences*
    or *paraphrases* are generated/retrieved.

    Given the complexity and potential ambiguity in replicating the exact augmentation
    that led to perfect/near-perfect scores[cite: 285, 280], we will implement a simplified
    version where we append the extracted keywords to the text. This boosts the signal
    of these important terms for BERT.
    """
    if not keywords:
        return text
    expanded_text = text + " " + " ".join(keywords)
    return expanded_text


df['keywords'] = df['processed_text'].apply(lambda x: extract_keywords_for_doc(x, top_n=3))
df['augmented_text'] = df.apply(lambda row: contextual_expansion(row['processed_text'], row['keywords'], sentence_model), axis=1)

print("\nSample of augmented text:")
print(df[['processed_text', 'keywords', 'augmented_text']].head())




Sample of augmented text:
                                      processed_text                 keywords  \
0  do you have the skin of a 80 year old grandma ...     [year, worry, water]   
1  this is taking a grain of truth and extrapolat...   [youll, truth, taking]   
2  honestly my favorite thing about this is that ...    [water, thing, prove]   
3                source doesnt sound right to me idk   [source, sound, right]   
4  damn i saw a movie in which the old woman bath...  [woman, water, virgins]   

                                      augmented_text  
0  do you have the skin of a 80 year old grandma ...  
1  this is taking a grain of truth and extrapolat...  
2  honestly my favorite thing about this is that ...  
3  source doesnt sound right to me idk source sou...  
4  damn i saw a movie in which the old woman bath...  


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast

# Initialize BERT Tokenizer
MODEL_NAME = 'bert-base-uncased' # Or any other BERT model you prefer from the paper like RoBERTa, DistilBERT [cite: 175]
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

# Texts and Labels
texts = df['augmented_text'].tolist()
labels = df[LABEL_COLUMN].tolist()

# Ensure labels are integers
labels = [int(label) for label in labels]


# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42, stratify=labels)

# Tokenize the texts
# The paper uses a max sequence length. BERT has a limit (typically 512).
# Let's find a suitable max_length or truncate.
# For now, let's set a common max_length.
MAX_LENGTH = 128 # Adjust based on your data's text length distribution

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors='pt')

print(f"\nShape of training input_ids: {train_encodings['input_ids'].shape}")
print(f"Shape of validation input_ids: {val_encodings['input_ids'].shape}")


Shape of training input_ids: torch.Size([5244, 128])
Shape of validation input_ids: torch.Size([1311, 128])


In [ ]:
import torch

class MisogynyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long) # BERT for sequence classification expects 'labels'
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MisogynyDataset(train_encodings, train_labels)
val_dataset = MisogynyDataset(val_encodings, val_labels)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True) # Adjust batch_size based on your GPU memory
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
from torch.optim import AdamW
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import numpy as np

# Load BERT model for sequence classification
# num_labels should be 2 for binary classification (misogynistic vs. non-misogynistic)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Set up training parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5) # Learning rate, 5e-5 is a common default for BERT

num_epochs = 3 # The paper doesn't specify epochs, 2-4 is common for fine-tuning
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
print("\nStarting training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Gradient clipping
        optimizer.step()
        lr_scheduler.step()

        if (batch_idx + 1) % 50 == 0: # Print progress every 50 batches
            print(f"Epoch {epoch + 1}/{num_epochs}, Batch {batch_idx + 1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} finished. Average Training Loss: {avg_train_loss:.4f}")

    # Validation step (optional, but good practice)
    model.eval()
    val_preds = []
    val_true_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            val_preds.extend(predictions.cpu().numpy())
            val_true_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(val_true_labels, val_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(val_true_labels, val_preds, average='binary') # Use 'binary' for 2 classes
    print(f"Validation Accuracy: {val_accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

print("Training complete.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Starting training...
Epoch 1/3, Batch 50/328, Loss: 0.0969
Epoch 1/3, Batch 100/328, Loss: 0.1250
Epoch 1/3, Batch 150/328, Loss: 0.4716
Epoch 1/3, Batch 200/328, Loss: 0.2407
Epoch 1/3, Batch 250/328, Loss: 0.2333
Epoch 1/3, Batch 300/328, Loss: 0.3681
Epoch 1 finished. Average Training Loss: 0.2800
Validation Accuracy: 0.9130, Precision: 0.9333, Recall: 0.2000, F1: 0.3294
Epoch 2/3, Batch 50/328, Loss: 0.1144
Epoch 2/3, Batch 100/328, Loss: 0.3817
Epoch 2/3, Batch 150/328, Loss: 0.1785
Epoch 2/3, Batch 200/328, Loss: 0.0093
Epoch 2/3, Batch 250/328, Loss: 0.3472
Epoch 2/3, Batch 300/328, Loss: 0.3843
Epoch 2 finished. Average Training Loss: 0.1576
Validation Accuracy: 0.9275, Precision: 0.7184, Recall: 0.5286, F1: 0.6091
Epoch 3/3, Batch 50/328, Loss: 0.4282
Epoch 3/3, Batch 100/328, Loss: 0.5315
Epoch 3/3, Batch 150/328, Loss: 0.0087
Epoch 3/3, Batch 200/328, Loss: 0.0009
Epoch 3/3, Batch 250/328, Loss: 0.0147
Epoch 3/3, Batch 300/328, Loss: 0.0199
Epoch 3 finished. Average Trainin

In [ ]:
# Evaluation has been partially done in the training loop's validation step.
# Here's a more formal evaluation function:

def evaluate_model(model, data_loader, device):
    model.eval()
    predictions_list = []
    true_labels_list = []
    probabilities_list = [] # For ROC AUC

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1)
            probs = torch.softmax(logits, dim=-1)[:, 1] # Probability of the positive class (class 1)

            predictions_list.extend(preds.cpu().numpy())
            true_labels_list.extend(labels.cpu().numpy())
            probabilities_list.extend(probs.cpu().numpy())

    accuracy = accuracy_score(true_labels_list, predictions_list)
    # For binary classification, specify pos_label=1 if your positive class is 1
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels_list, predictions_list, average='binary', pos_label=1, zero_division=0)

    try:
        roc_auc = roc_auc_score(true_labels_list, probabilities_list)
    except ValueError as e:
        print(f"ROC AUC calculation error: {e}. This might happen if only one class is present in true labels during a batch or small evaluation set.")
        roc_auc = 0.0 # Or handle as appropriate

    print(f"\n--- Final Evaluation Metrics ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f} [cite: 230]")
    print(f"Recall: {recall:.4f} [cite: 231]")
    print(f"F1-Score: {f1:.4f} [cite: 234]")
    print(f"ROC AUC: {roc_auc:.4f} [cite: 238]")

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc
    }

# Evaluate on the validation set
evaluation_results = evaluate_model(model, val_loader, device)


--- Final Evaluation Metrics ---
Accuracy: 0.9252
Precision: 0.6842 [cite: 230]
Recall: 0.5571 [cite: 231]
F1-Score: 0.6142 [cite: 234]
ROC AUC: 0.8880 [cite: 238]
